In [17]:
import os
import sys
import pandas as pd
import argparse
from datetime import datetime, timedelta
import pathlib

# 新增：將上一層目錄加入 sys.path
parent_path = pathlib.Path().absolute().parent
sys.path.insert(0, str(parent_path))
# Setup Django environment
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'website_configs.settings')
import django
django.setup()
# 重要：設定環境變數以允許在 Jupyter 的異步環境中執行同步操作
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Now we can import Django models
from app_user_keyword_db.models import NewsData

In [18]:
from django.db.models import Q, Max, F
from collections import Counter

In [19]:
# Searching keywords from "content" column
# This function now uses database queries instead of pandas
def filter_database_fullText(user_keywords, cond, cate, weeks):
    # Get the latest date in the database
    latest_date = NewsData.objects.aggregate(max_date=Max('date'))['max_date']
    
    # Calculate start date
    start_date = latest_date - timedelta(weeks=weeks)
    
    # Base query - filter by date range
    queryset = NewsData.objects.filter(date__gte=start_date, date__lte=latest_date)
    
    # Filter by category if not "全部"
    if cate != "全部":
        queryset = queryset.filter(category=cate)
    
    # Filter by keywords based on condition (AND or OR)
    if cond == 'and':
        # For AND condition, we need all keywords to be present
        for kw in user_keywords:
            queryset = queryset.filter(content__contains=kw)
    elif cond == 'or':
        # For OR condition, any keyword can be present
        q_objects = Q()
        for kw in user_keywords:
            q_objects |= Q(content__contains=kw)
        queryset = queryset.filter(q_objects)
    
    return queryset

In [20]:

user_keywords = ['川普', '大谷翔平']  # Example keywords
cond = 'and'  # Example condition (and/or), 
cate = '全部'  # Example category (or "全部" for all categories)
weeks = 4  # Example weeks
queryset = filter_database_fullText(user_keywords, cond, cate, weeks)

In [21]:
for news in queryset:
    print(news.title)

挑戰完全打擊卻選到保送　道奇總仔讚大谷翔平：以團隊為重
錯過7億鎂大谷　藍鳥搬出史上第三大合約綁住小葛雷諾
道奇進白宮會見川普 大谷翔平與美國獨立宣言合照
MLB》川普在白宮表揚道奇 握手稱讚大谷翔平「宛如電影明星」
大谷翔平開轟差一點完全打擊！宇宙道奇本季首度二連敗


In [22]:
# Limit to k results and get specific fields
news_items = queryset.values('category', 'title', 'link', 'photo_link')[:3]

In [23]:
news_items

<QuerySet [{'category': '運動', 'title': '挑戰完全打擊卻選到保送\u3000道奇總仔讚大谷翔平：以團隊為重', 'link': 'https://tw.news.yahoo.com/%E6%8C%91%E6%88%B0%E5%AE%8C%E5%85%A8%E6%89%93%E6%93%8A%E5%8D%BB%E9%81%B8%E5%88%B0%E4%BF%9D%E9%80%81-%E9%81%93%E5%A5%87%E7%B8%BD%E4%BB%94%E8%AE%9A%E5%A4%A7%E8%B0%B7%E7%BF%94%E5%B9%B3-%E4%BB%A5%E5%9C%98%E9%9A%8A%E7%82%BA%E9%87%8D-043253712.html', 'photo_link': 'https://s.yimg.com/ny/api/res/1.2/jdKWh.T4GM3o0mACXry5qA--/YXBwaWQ9aGlnaGxhbmRlcjt3PTEyMDA7aD04MDA-/https://media.zenfs.com/en/gotv_ctitv_com_tw_678/cbfe92b7af5e52edb77183ef96d447b0'}, {'category': '運動', 'title': '錯過7億鎂大谷\u3000藍鳥搬出史上第三大合約綁住小葛雷諾', 'link': 'https://tw.news.yahoo.com/%E9%8C%AF%E9%81%8E7%E5%84%84%E9%8E%82%E5%A4%A7%E8%B0%B7-%E8%97%8D%E9%B3%A5%E6%90%AC%E5%87%BA%E5%8F%B2%E4%B8%8A%E7%AC%AC%E4%B8%89%E5%A4%A7%E5%90%88%E7%B4%84%E7%B6%81%E4%BD%8F%E5%B0%8F%E8%91%9B%E9%9B%B7%E8%AB%BE-061836382.html', 'photo_link': 'https://media.zenfs.com/en/taisounds_com_380/5af69b48d3a41d91f5b2d6fc9fd45a05'}, {'category': '運動', 'title

In [24]:
dates = list(queryset.values_list('date', flat=True))